In [15]:
# Import TF and TF Hub libraries.
import tensorflow as tf
import tensorflow_hub as hub
import tf2onnx
import subprocess
print(tf2onnx.__version__)

# Load the input image
image_path = './kyokushi-nanaya.png'
image = tf.io.read_file(image_path)
image = tf.compat.v1.image.decode_jpeg(image)
image = tf.expand_dims(image, axis=0)
# Resize and pad the image to keep the aspect ratio and fit the expected size.
image = tf.cast(tf.image.resize_with_pad(image, 192, 192), dtype=tf.int32)

# Download the model from TF Hub.
model = hub.load("https://www.kaggle.com/models/google/movenet/TensorFlow2/singlepose-lightning/4")
movenet = model.signatures['serving_default']

# Run model inference.
outputs = movenet(image)
# Output is a [1, 1, 17, 3] tensor.
keypoints = outputs['output_0']
print(keypoints)

tf.saved_model.save(model, "./movenet_savedmodel", signatures=model.signatures)

result = subprocess.run(["python", "-m", "tf2onnx.convert", "--saved-model", "./movenet_savedmodel", "--output", "./movenet.onnx", "--opset", "14"], capture_output=True, text=True)

print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

1.16.1
tf.Tensor(
[[[[0.36727607 0.5812892  0.3382075 ]
   [0.34428078 0.60589564 0.41563293]
   [0.34039626 0.5655281  0.38113177]
   [0.35466823 0.6403011  0.40432182]
   [0.3364695  0.5375261  0.3753358 ]
   [0.47575623 0.67586076 0.815754  ]
   [0.44372225 0.47061938 0.6161275 ]
   [0.59732735 0.55252963 0.25787717]
   [0.5347773  0.3918764  0.2246131 ]
   [0.5000789  0.3584857  0.5461282 ]
   [0.4955294  0.3355862  0.3807939 ]
   [0.7746345  0.5625905  0.49802983]
   [0.74924916 0.4175235  0.5020223 ]
   [0.7600435  0.545617   0.14871527]
   [0.732331   0.3160424  0.17471263]
   [0.7236688  0.29664153 0.28401735]
   [0.7417898  0.30024636 0.2414978 ]]]], shape=(1, 1, 17, 3), dtype=float32)
INFO:tensorflow:Assets written to: ./movenet_savedmodel\assets


INFO:tensorflow:Assets written to: ./movenet_savedmodel\assets


STDOUT: 
STDERR: WARNING:tensorflow:From c:\Users\mahd\AppData\Local\Programs\Python\Python311\Lib\site-packages\tf2onnx\tf_loader.py:68: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.


<frozen runpy>:128: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
2025-07-30 20:02:58,299 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2025-07-30 20:02:58,304 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2025-07-30 20:03:43,966 - INFO - Signatures found in model: [serving_default].
2025-07-30 20:03:43,966 - WARNING - '--signature_def' not specified, using first signature: serving_default
2025-07-30 20:03:43,966 - INFO - Output names: ['output_0']
I0000 00:00:1753923824.145332   2630